# Train

In [18]:
from datasets import load_dataset
import random
random.seed(42)


raw_datasets = load_dataset("TinyStoriesV2_SpecialTokens", split='train').shuffle(seed=42).select(range(100000))

print(f'{raw_datasets["text"][0][:-13]} <|end_story|>')

<|start_story|>Anna and Ben are playing with their toys in the living room. Anna has a lot of dolls and Ben has a lot of cars. They like to count their toys and see who has more.
"I have one, two, three, four, five, six dolls!" Anna says proudly.
"I have one, two, three, four, five, six, seven cars!" Ben says, showing off his cars.
"Wow, you have more cars than me!" Anna says, feeling a little sad.
"But you have more dolls than me!" Ben says, trying to cheer her up.
They hug and smile. They are happy to share their toys.
But then, they hear a noise from the kitchen. It is their cat, Luna. Luna is very nosy. She likes to explore and touch everything. She sees a bowl of cookies on the table. She jumps on the table and tries to eat a cookie.
"Hey, that's our snack!" Anna says, running to the kitchen.
"Stop, Luna, stop!" Ben says, following Anna.
They reach the kitchen and see Luna with a cookie in her mouth. She looks guilty and runs away.
"Oh no, she ate one of our cookies!" Anna says, l

In [19]:
from tqdm import tqdm

training_corpus = [
    f"{text[:-13]} " for i in tqdm(range(0, len(raw_datasets["text"]), 4096)) for text in raw_datasets["text"][i:i+4096]
]

100%|██████████| 25/25 [00:09<00:00,  2.54it/s]


In [20]:
from transformers import AutoTokenizer, LlamaTokenizer
old_tokenizer = AutoTokenizer.from_pretrained("tokenizer_draft_llama")

print(old_tokenizer)

example = """<|start_story|>Once upon a time, in a big forest, there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, rocks, and hills. One day, Roxy found an icy hill. She had never seen anything like it before. It was shiny and cold, and she wanted to climb it. Roxy tried to climb the icy hill, but it was very slippery. She tried again and again, but she kept falling down. Roxy was sad. She wanted to climb the icy hill so much. Then, she saw a little bird named Billy. Billy saw that Roxy was sad and asked, "Why are you sad, Roxy?" Roxy told Billy about the icy hill and how she couldn't climb it. Billy said, "I have an idea! Let's find some big leaves to put under your feet. They will help you climb the icy hill." Roxy and Billy looked for big leaves and found some. Roxy put the leaves under her feet and tried to climb the icy hill again. This time, Roxy didn't slip. She climbed and climbed until she reached the top of the icy hill. Roxy was so happy! She and Billy played on the icy hill all day. From that day on, Roxy and Billy were the best of friends, and they climbed and played together all the time. And Roxy learned that with a little help from a friend, she could climb anything. <|end_story|>"""
tokens = old_tokenizer.tokenize(example)
print(tokens)
print(len(tokens))

LlamaTokenizerFast(name_or_path='tokenizer_draft_llama', vocab_size=32000, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<|start_story|>', 'eos_token': '<|end_story|>', 'unk_token': '<unk>', 'pad_token': '<unk>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<|start_story|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("<|end_story|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
['▁<|start_story|>', 'Once', '▁upon', '▁a', '▁time', ',', '▁in', '▁a', '▁big', '▁forest', ',', '▁there', '▁lived', '▁a', '▁r', 'hin', 'oc', 'eros', '▁named', '▁Ro', 'xy', '.', '▁Ro', 'xy', '▁loved', '▁to', '▁clim', 'b', '.', '▁She', '▁clim', 'bed', '▁trees', ',', '▁rocks', ',', '▁and', '▁hills', '.', 

In [22]:
new_tokenizer_draft = AutoTokenizer.from_pretrained("tokenizer_empty_llama")

print()

example = """<|start_story|>Once upon a time, in a big forest, there lived a rhinoceros named Roxy. <|end_story|>"""
tokens = new_tokenizer_draft.tokenize(example)
print(tokens)
print(len(tokens))


['▁<|start_story|>', '<unk>', '▁<|end_story|>']
3


In [23]:
tokenizer = new_tokenizer_draft.train_new_from_iterator(tqdm(iter(training_corpus)), 2048)

100000it [00:00, 114472.14it/s]


In [24]:
example = """<|start_story|>Once upon a time, in a big forest, there lived a rhinoceros named Roxy. Roxy loved to climb. She climbed trees, rocks, and hills. One day, Roxy found an icy hill. She had never seen anything like it before. It was shiny and cold, and she wanted to climb it. Roxy tried to climb the icy hill, but it was very slippery. She tried again and again, but she kept falling down. Roxy was sad. She wanted to climb the icy hill so much. Then, she saw a little bird named Billy. Billy saw that Roxy was sad and asked, "Why are you sad, Roxy?" Roxy told Billy about the icy hill and how she couldn't climb it. Billy said, "I have an idea! Let's find some big leaves to put under your feet. They will help you climb the icy hill." Roxy and Billy looked for big leaves and found some. Roxy put the leaves under her feet and tried to climb the icy hill again. This time, Roxy didn't slip. She climbed and climbed until she reached the top of the icy hill. Roxy was so happy! She and Billy played on the icy hill all day. From that day on, Roxy and Billy were the best of friends, and they climbed and played together all the time. And Roxy learned that with a little help from a friend, she could climb anything. <|end_story|>"""

tokens = tokenizer.tokenize(example)
print(tokens)
print(len(tokens))

['▁<|start_story|>', 'On', 'ce▁', 'upon▁a▁tim', 'e,▁', 'in▁a▁', 'big▁', 'forest', ',▁there▁lived▁a▁', 'r', 'h', 'in', 'o', 'c', 'er', 'o', 's▁', 'named▁', 'R', 'o', 'x', 'y.▁', 'R', 'o', 'x', 'y▁', 'loved▁to▁', 'climb', '.▁She▁', 'climbed▁', 'tre', 'es,▁', 'rock', 's,▁and▁', 'hill', 's.▁One▁day,▁', 'R', 'o', 'x', 'y▁', 'found▁', 'an▁', 'ic', 'y▁', 'hill', '.▁She▁', 'had▁', 'never▁', 'se', 'en', '▁any', 'thing▁', 'like▁', 'it▁', 'befor', 'e', '.▁It▁was▁', 'shin', 'y▁and▁', 'co', 'ld', ',▁and▁', 'she▁', 'wanted▁to▁', 'climb', '▁', 'it', '.▁', 'R', 'o', 'x', 'y▁', 'tried▁to▁', 'climb', '▁the▁', 'ic', 'y▁', 'hill', ',▁but▁it▁', 'was▁very▁', 's', 'li', 'pp', 'er', 'y.▁She▁', 'tri', 'ed▁a', 'gain', '▁and▁', 'again', ',▁but▁she▁', 'kept▁', 'fall', 'ing▁', 'down', '.▁', 'R', 'o', 'x', 'y▁', 'was▁sad', '.▁She▁wanted▁to▁', 'climb', '▁the▁', 'ic', 'y▁', 'h', 'ill▁', 'so▁', 'much', '.▁Then,▁', 'she▁', 'saw▁a▁', 'little▁bird▁', 'named▁', 'B', 'ill', 'y.▁', 'B', 'ill', 'y▁', 'saw▁', 'that▁', 'R', 'o

In [25]:
tokenizer.save_pretrained("story_tokenizer_2048")

('story_tokenizer_2048\\tokenizer_config.json',
 'story_tokenizer_2048\\special_tokens_map.json',
 'story_tokenizer_2048\\tokenizer.json')